# Introduction
<hr style = "border:2px solid black" ></hr>


**What?** Linear programming Scipy vs. CVXPY



# Imports
<hr style = "border:2px solid black" ></hr>

In [6]:
from scipy.optimize import linprog
from time import time
import cvxpy as cp 

# Linear programming problem
<hr style = "border:2px solid black" ></hr>


- Suppose the following LP problem:

```
f = -1x[0] + 4x[1]
 
-3x[0] + 1x[1] <= 6
1x[0] + 2x[1] <= 4
          x[1] >= -3
-inf <= x[0] <= inf
```



# Scipy-Optimize
<hr style = "border:2px solid black" ></hr>


- In scipy, the linear programming is implemented in linprog module. 
- There are two methods: `simplex` and `interior-point`. 
- The simplex method is more like a brute-force search algorithm, it has high complexity, therefore not suitable for high dimensional problem. 
- The `interior-point` method is makeing the problem into primal-dual formulation and place a log barrier for the inequality constraints. 



In [2]:
c = [-1, 4]
A = [[-3, 1], [1, 2]]
b = [6, 4]

x0_bounds = (None, None)
x1_bounds = (-3, None)

# The default method is the ``simplex``. It is working good for low dimensional problem.
#
# Constraint is A x <= b


res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds),
              options={"disp": True})

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -8.0                
0.09885158404625    0.09885158404625    0.09885158404625    0.903461537018   0.09885158404625    -6.284698425658     
0.05788429348353    0.05788429348355    0.05788429348355    0.4273037994111  0.05788429348355    -7.864724729573     
0.04539867008243    0.04539867008244    0.04539867008244    0.2387091287399  0.04539867008244    -12.78916804766     
0.00666151448168    0.006661514481681   0.006661514481682   0.8665142913493  0.006661514481682   -21.3520715063      
6.299626472829e-06  6.299626472597e-06  6.299626472583e-06  1.0              6.299626472568e-06  -21.99681708159     
3.150184161152e-10  3.150192895998e-10  3.150192773305e-10  0.9999499939736  3.150193133197e-10  -21.99999984082     
Optimization terminated successfully.
         Current fu

In [3]:
# If we try a random objective function with slightly higher dimension n =40

n = 40
m = 3

c = np.ones(n)
A = np.random.random([m, n])
b = -np.ones(m)

bounds = np.array([[-1, 1]] * n)


start = time()

res = linprog(c, A_ub=A, b_ub=b, bounds=bounds,
              options={"disp": True})

end = time()
print('elapsed time is', end-start, 'seconds')

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 0.0                 
0.3484953214028     0.3484953214028     0.3484953214028     0.6594789167432  0.3484953214028     -6.25344838669      
0.1818499603898     0.1818499603901     0.1818499603901     0.5094733134214  0.1818499603901     -26.09989739784     
0.01369102576924    0.01369102576927    0.01369102576927    0.9282523323965  0.01369102576927    -38.67896382329     
5.714386370907e-06  5.714386365036e-06  5.714386365032e-06  0.9995858615637  5.714386365035e-06  -39.99964246769     
2.857116028198e-10  2.857197356541e-10  2.857197356534e-10  0.999949999927   2.857197356539e-10  -39.99999998212     
1.864590841776e-14  1.428612222168e-14  1.428598678273e-14  0.99995          1.428598678275e-14  -40.0               
Optimization terminated successfully.
         Current fu

In [4]:
# Using the interior method is faster than the simplex method.
start = time()
res = linprog(c, A_ub=A, b_ub=b, bounds=bounds,
              method='interior-point', options={"disp": True})

end = time()
print('elapsed time is', end-start, 'seconds')

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 0.0                 
0.3484953214028     0.3484953214028     0.3484953214028     0.6594789167432  0.3484953214028     -6.25344838669      
0.1818499603898     0.1818499603901     0.1818499603901     0.5094733134214  0.1818499603901     -26.09989739784     
0.01369102576924    0.01369102576927    0.01369102576927    0.9282523323965  0.01369102576927    -38.67896382329     
5.714386370907e-06  5.714386365036e-06  5.714386365032e-06  0.9995858615637  5.714386365035e-06  -39.99964246769     
2.857116028198e-10  2.857197356541e-10  2.857197356534e-10  0.999949999927   2.857197356539e-10  -39.99999998212     
1.864590841776e-14  1.428612222168e-14  1.428598678273e-14  0.99995          1.428598678275e-14  -40.0               
Optimization terminated successfully.
         Current fu

# CVXPY
<hr style = "border:2px solid black" ></hr>

In [7]:
# The solution is solved through SQP method

x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(c.T@x),
                  [A@x <= b,
                  x <= 1,
                  x >= -1])
start = time()
prob.solve(verbose=True)
end = time()
print('elapsed time is', end-start, 'seconds')

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Sep 08 09:57:20 PM: Your problem has 40 variables, 3 constraints, and 0 parameters.
(CVXPY) Sep 08 09:57:20 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 08 09:57:20 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 08 09:57:20 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 08 09:57:20 PM: Compiling problem (target solver=ECOS).
(CVXPY) Sep 08 09:57:20 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

# References
<hr style = "border:2px solid black" ></hr>


- https://colab.research.google.com/drive/1bFcr1J0a6Euwp09vY_IID-LRxCKCdm_I#scrollTo=7vtqCZnN5-Sz

